# <span style="color:darkred">Multiplicative Schwarz  using multiple strategies</span>.

<img align="left" src="Schwarz.jpg" width=120 height=120 />

### [Simon Tavener](https://www.math.colostate.edu/~tavener/) and Andre Leautaud

#### Full text for this notebook can be found at the following [link](https://www.math.colostate.edu/~tavener/andre/DD_multiphysics.pdf)

#### Set path and parameters

In [275]:
import numpy as np
import math
%run DD_Utilities_sjt_al.ipynb
%run MS_Utilities_sjt_al.ipynb
iprint= 4

In [276]:
iparam= 4

In [277]:
if iparam == 0:
    n= 10
    p= 4 
    b= [1,2,4,7]
    e= [3,4,8,10] #List of ends (inclusive) or integer representing overlap
    K= 3
    if K > 500:
        raise ValueError("K is too large")

In [278]:
# This example shows how to create domains that each overlap by 1 column
#small system
if iparam == 1:
    n= 8
    p= 3 
    b= [1,3,5]
    e= 1
    K= 3
    if K > 500:
        raise ValueError("K is too large")

In [279]:
# This example shows the optional parameters for set_parameters
if iparam == 2:
    n= 12
    p= -1 #p should be a positive integer or -1
    b= [-1] # an evenly dispersed set of domains will be chosen
    e= [-1] #List of ends or integer representing overlap or [-1] to automaically choose overlap
    K= 6
    if K > 500:
        raise ValueError("K is too large")

In [280]:
#subdomains contained in others
if iparam == 3:
    n= 10
    p= 5 #p should be a positive integer or -1
    b= [1,1,5,5,7]
    e= [2,4,6,8,10] 
    K= 6
    if K > 500:
        raise ValueError("K is too large")

In [281]:
#overlap does not connect all subdomains
if iparam == 4:
    n= 9
    p= 4 
    b= [1,3,6,8]
    e= [2,5,7,9]
    K= 3
    if K > 500:
        raise ValueError("K is too large")

In [282]:
p,b,e= set_parameters(n,p,b,e)  #set_parameters creates a sample set of domains if the user does not fully specify it.
print(p,b,e)
nn,nb,ne,index= fix_domains(n,p,b,e,iprint)

4 [1, 3, 6, 8] [2, 5, 7, 9]


#### Establish and solve forward system 

In [283]:
sread= False    #Read system of equations from csv files
swrite= True  #Write system to csv file

M,x,y,xstore= Linear_system(n,sread,swrite,iparam)
svpert=Make_svpert_mult(n,p,K,sread,iparam)
if iprint >= 4:
    print(M)
    print('y')
    print(y)        
    print('x')
    print(x)

if swrite:
    save_to_csv(n,p,K,nb,nn,ne,M,y,svpert,iparam)

[[9.99017899e+00 6.05181485e-01 8.33524885e-01 7.73481201e-01
  4.16700431e-01 4.88442909e-01 1.77210291e-01 9.25269108e-01
  4.99552610e-01]
 [6.00490233e-01 9.77928079e+00 2.90909275e-01 4.46620190e-01
  3.84136988e-01 6.31283826e-01 1.74056683e-01 4.78664975e-01
  4.30488448e-01]
 [2.74415429e-01 3.90222908e-01 9.43635498e+00 6.06350353e-01
  6.46611944e-01 4.56710039e-01 1.09057222e-01 7.33592985e-01
  4.29689015e-01]
 [9.78885980e-01 6.74799941e-01 7.17597291e-01 9.31444727e+00
  7.64163484e-01 6.04144626e-01 3.61625515e-02 6.32094987e-01
  8.18111701e-02]
 [1.01419543e-01 5.22527149e-01 7.54919551e-01 6.13157078e-01
  9.52046993e+00 1.07644651e-01 2.36722304e-01 5.52369451e-02
  5.04975667e-01]
 [8.18227701e-01 7.15473266e-01 4.37690765e-01 2.25538828e-01
  1.60475876e-01 9.96005539e+00 9.48428389e-01 7.22139454e-01
  6.76493569e-01]
 [7.46666630e-01 5.69892030e-01 8.78724969e-01 3.24473937e-02
  2.79757696e-01 4.96295065e-01 9.90561062e+00 3.52165440e-01
  6.82498072e-01]
 [9.80

### Iterative strategy
$u^{\{k\}}= u^{\{k-1\}} + Br = u^{\{k-1\}} + B(f-Au^{\{k-1\}})$, $\quad k=1, \dots, K,$ where $B$ is an approximation to $A^{-1}$

<ins>Multiplicative<ins>
$A_i= R_i A R_i^\top \in \mathbb{R}^{m_i \times m_i}, \\
B_i= R_i^\top A_i^{-1} R_i  = R_i^\top ( R_i A R_i^\top)^{-1} R_i  \in \mathbb{R}^{n \times n}, \\
C_i= B_i A   \in \mathbb{R}^{n \times n},  \\
D_i= (I-C_i) \in \mathbb{R}^{n \times n},  \\
f_i= B_i f   \in \mathbb{R}^{n}, \; i=1,\dots,p.
$

In [284]:
#Construct domains, R,A,B,C and f
R= Rmatrices(n,p,nn,index,iprint)
A= Amatrices(M,R,n,p,nn,iprint)
B= Bmatrices(A,R,n,p,nn,iprint)
C= Cmatrices(M,B,n,p,iprint) 
D= Dmatrices_multiplicative(C,n,p,iprint)
f= fvector(B,y,n,p,iprint)

Print DD matrix 
[[[ 0.00000000e+00  7.60063157e-18 -8.19371795e-02 -7.49363176e-02
   -3.94783241e-02 -4.51497727e-02 -1.67224609e-02 -8.99875154e-02
   -4.75144556e-02]
  [-4.16510881e-19 -2.22044605e-16 -2.47162143e-02 -4.10686299e-02
   -3.68565591e-02 -6.17808039e-02 -1.67716842e-02 -4.34212250e-02
   -4.11028674e-02]
  [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000

#### Perform $K$  iterations computing subdomain contributions independently
$
u^{\{k+i/p\}} = u^{\{k+(i-1)/p\}} + B_i(f-Au^{\{k+(i-1)/p\}}) =D_i u^{\{k+(i-1)/p\}} + f_i \quad i=1,\dots,p$

In [285]:
#Perform D iteration
vstore= Diteration_multiplicative(D,f,n,p,K,iprint)
if iprint >= 4:
    print('vstore')
    print(vstore)


Outer iteration k, Inner iteration i 0 0
[[0.04858043 0.06337318 0.         0.         0.         0.
  0.         0.         0.        ]]
Outer iteration k, Inner iteration i 0 1
[[0.04858043 0.06337318 0.03213781 0.06875226 0.09123279 0.
  0.         0.         0.        ]]
Outer iteration k, Inner iteration i 0 2
[[0.04858043 0.06337318 0.03213781 0.06875226 0.09123279 0.01840509
  0.01040731 0.         0.        ]]
Outer iteration k, Inner iteration i 0 3
[[ 0.04858043  0.06337318  0.03213781  0.06875226  0.09123279  0.01840509
   0.01040731  0.03829968 -0.01376077]]
Outer iteration 0
[[ 0.04858043  0.06337318  0.03213781  0.06875226  0.09123279  0.01840509
   0.01040731  0.03829968 -0.01376077]]
Outer iteration k, Inner iteration i 1 0
[[ 0.03339571  0.05398373  0.03213781  0.06875226  0.09123279  0.01840509
   0.01040731  0.03829968 -0.01376077]]
Outer iteration k, Inner iteration i 1 1
[[ 0.03339571  0.05398373  0.02962217  0.06742746  0.09223548  0.01840509
   0.01040731  0.0382

#### Perform $K$ iterations using iteration matrices $S$ and $T$,  and  which compute the solution in all $p$ domains simultanously
Let $p=5$ and construct S and T, g and v such that

$S = \pmatrix{
 I   &  0   &   0   &  0   &   0   \cr
-D_2 &  I   &   0   &  0   &   0   \cr
 0   & -D_3 &   I   &  0   &   0   \cr
 0   &   0  &  -D_4 &  I   &   0   \cr
 0   &   0  &   0   & -D_5 &   I
} \in \mathbb{R}^{np \times np}$,
$T =
\pmatrix{
 0 & 0 & 0 & 0 & -D_1  \cr
 0 & 0 & 0 & 0 &  0    \cr
 0 & 0 & 0 & 0 &  0    \cr
 0 & 0 & 0 & 0 &  0    \cr
 0 & 0 & 0 & 0 &  0
} \in \mathbb{R}^{np \times np}$, 
$g =
\pmatrix{
f_1  \cr
f_2  \cr
f_3  \cr
f_4  \cr
f_5
}
\in \mathbb{R}^{np}$,
$v^{\{k\}} =
\pmatrix{
u^{\{(k-1)+1/5\}}  \cr
u^{\{(k-1)+2/5\}}  \cr
u^{\{(k-1)+3/5\}}  \cr
u^{\{(k-1)+4/5\}}  \cr
u^{\{k\}}
}
\in \mathbb{R}^{np}$,

In [286]:
# Construct S, T and g and solve
S= Smatrix_multiplicative(D,n,p,iprint)
T= Tmatrix_multiplicative(D,n,p,iprint)
g= gvector_multiplicative(f,n,p,iprint)

wstore= STiteration(S,T,g,n,p,K,iprint)
if iprint >= 4:
    print('wstore')
    print(wstore)

Print S matrix 

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
Print T matrix
[[ 0.          0.          0.         ...  0.01672246  0.08998752
   0.04751446]
 [ 0.          0.          0.         ...  0.01677168  0.04342122
   0.04110287]
 [ 0.          0.          0.         ... -0.         -0.
  -0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
Print g vector
[[0.04858043]
 [0.06337318]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.03536502]
 [0.07794202]
 [0.09438076]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.03020962]
 

#### Perform $K$ iterations as single matrix solve

$K=6$ iterations of multiplicative Schwarz can be written as the $npK$-dimensional system of equations $Uz=h$, where

$U=
\pmatrix{
 S & 0 & 0 & 0 & 0 & 0   \cr
 T & S & 0 & 0 & 0 & 0   \cr
 0 & T & S & 0 & 0 & 0   \cr
 0 & 0 & T & S & 0 & 0   \cr
 0 & 0 & 0 & T & S & 0   \cr
 0 & 0 & 0 & 0 & T & S
}
$
,
$z=
\pmatrix{
v^{\{1\}}  \cr
v^{\{2\}}  \cr
v^{\{3\}}  \cr
v^{\{4\}}  \cr
v^{\{5\}}  \cr
v^{\{6\}}
}
$
,
$
h=
\pmatrix{
g  \cr
g  \cr
g  \cr
g  \cr
g  \cr
g
}
$
.

In [287]:
# Construct U and h and solve
U= Umatrix_multiplicative(S,T,n,p,K,iprint) 
h= hvector_multiplicative(g,n,p,K,iprint)

# Solve using U and h
zstore= Uhsolve_multiplicative(U,h,n,p,K,iprint);
if iprint >= 4:
    print('zstore')
    print(zstore)

h vector
[[0.04858043]
 [0.06337318]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.03536502]
 [0.07794202]
 [0.09438076]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.03020962]
 [0.02277656]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.05987377]
 [0.0038687 ]
 [0.04858043]
 [0.06337318]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.03536502]
 [0.07794202]
 [0.09438076]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.03020962]
 [0.02277656]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.0598737

#### Compare Solutions from all three methods

In [288]:
solution_compare(xstore,vstore,wstore,zstore,iprint)

norm(exact - iterative) solutions    =  4.912657e-05
Difference between vstore and wstore =  4.354121e-17
Difference between wstore and zstore =  4.670886e-17
Difference between vstore and zstore =  6.598793e-17


### Add discretization error and compute error estimates and effectivity ratios

#### $K$  iterations computing subdomain contributions independently

In [289]:
# Set adjoint RHS
psi= np.ones((n,1))

# Perform D iteration with error: add random error to each subdomain solve
v_store,va_store,r_store= Diteration_approx(D,f,n,p,K,iprint,svpert)
if iprint >= 6:
    print(v_store)
    print(va_store)
    print(rstore)


Outer iteration   0
Outer iteration   0, Inner iteration   0
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.06404515]
 [ 0.06498574]
 [ 0.00167241]
 [ 0.00593789]
 [ 0.00610496]
 [ 0.01019775]
 [-0.01007681]
 [ 0.02199571]
 [-0.00238761]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.   

#### Calculate the discretization error

In [290]:
# Perform adjoint solve 
phi_store= Diteration_adjoint(D,psi,n,p,K,iprint)

# Compute error estimate as (r,phi)
qoi_discretization_error_estimate= r_store.T @ phi_store

# Compute error and error in QoI directly
vesoln=  v_store[n*p*K+n*(p-1):n*p*(K+1),0:1]
vsoln=  va_store[n*p*K+n*(p-1):n*p*(K+1),0:1]
verror= vesoln-vsoln

qoi_discretization_error= psi.T @ verror

k, i, pindex, id1, id2: 3 4 0 99 108
Diteration_adjoint: special case 

k, i, pindex, id1, id2: 3 3 3 90 99
[[0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.000000

#### Calculate the effectivity ratio

In [291]:
effectivity_ratio_disc = qoi_discretization_error_estimate/qoi_discretization_error
print(f'QoI_discretization_error          = {qoi_discretization_error[0,0]:13.6e}')
print(f'QoI_discretization_error_estimate = {qoi_discretization_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_disc[0,0]:13.3f}')

QoI_discretization_error          = -4.369509e-03
QoI_discretization_error_estimate = -4.369509e-03
Effectivity ratio                 =         1.000


#### Calculate the total error

In [292]:
total_error= xstore-vsoln
qoi_total_error= psi.T@total_error

phi= np.linalg.solve(M.T,psi)
r= y-M@vsoln
qoi_total_error_estimate= r.T@phi

#### Calculate the effectivity ratio

In [293]:
effectivity_ratio_total= qoi_total_error_estimate/qoi_total_error
print(f'QoI_total_error                   = {qoi_total_error[0,0]:13.6e}')
print(f'QoI_total_error_estimate          = {qoi_total_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_total[0,0]:13.3f}')

QoI_total_error                   = -4.317790e-03
QoI_total_error_estimate          = -4.317790e-03
Effectivity ratio                 =         1.000


#### Calculate the iteration error

In [294]:
qoi_iteration_error= qoi_total_error-qoi_discretization_error
print(f'QoI_iteration_error               = {qoi_iteration_error[0,0]:13.6e}')

QoI_iteration_error               =  5.171947e-05


#### $K$ iterations as single matrix solve

In [295]:
# Perform U iteration with error: add random error to linear system solution
zexact,zapprox,zresid= Uhsolve_multiplicative_approx(U,h,n,p,K,iprint,svpert)

# Adjoint data (Adjoint system is of size npK)
psi_npK= np.zeros((n*p*K,1))
psi_npK[n*p*K-n:n*p*K,0]= psi.reshape(n)

#  Solve adjoint equation
phi_npK= np.linalg.solve(U.T,psi_npK)
if iprint >= 2:
    print(f'Norm of difference between phi_npK and phi_store = {np.linalg.norm(phi_npK-phi_store):13.6e}')

# Compute error estimate
qoi_discretization_error_estimate= phi_npK.T @ zresid

# Compute error and error in QoI directly
xsoln= zexact[n*p*K-n:n*p*K,0:1]
zsoln= zapprox[n*p*K-n:n*p*K,0:1]
zerror= xsoln-zsoln
qoi_discretization_error= psi.T @zerror

[ 0.04858043  0.06337318  0.          0.          0.          0.
  0.          0.          0.          0.04858043  0.06337318  0.03213781
  0.06875226  0.09123279  0.          0.          0.          0.
  0.04858043  0.06337318  0.03213781  0.06875226  0.09123279  0.01840509
  0.01040731  0.          0.          0.04858043  0.06337318  0.03213781
  0.06875226  0.09123279  0.01840509  0.01040731  0.03829968 -0.01376077]
[ 0.03339571  0.05398373  0.03213781  0.06875226  0.09123279  0.01840509
  0.01040731  0.03829968 -0.01376077  0.03339571  0.05398373  0.02962217
  0.06742746  0.09223548  0.01840509  0.01040731  0.03829968 -0.01376077
  0.03339571  0.05398373  0.02962217  0.06742746  0.09223548  0.01846947
  0.01187455  0.03829968 -0.01376077  0.03339571  0.05398373  0.02962217
  0.06742746  0.09223548  0.01846947  0.01187455  0.04026508 -0.01233665]
[ 0.03338956  0.0538909   0.02962217  0.06742746  0.09223548  0.01846947
  0.01187455  0.04026508 -0.01233665  0.03338956  0.0538909   0.0

#### Calculate the effectivity ratio

In [296]:
effectivity_ratio_disc= qoi_discretization_error_estimate/qoi_discretization_error
print(f'QoI_discretization_error          = {qoi_discretization_error[0,0]:13.6e}' )
print(f'QoI_discretization_error_estimate = {qoi_discretization_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_disc[0,0]:13.3f}')

QoI_discretization_error          = -1.088356e-02
QoI_discretization_error_estimate = -1.088356e-02
Effectivity ratio                 =         1.000


#### Calculate the total error

In [297]:
total_error= xstore-zsoln
qoi_total_error= psi.T @ total_error

r= y-M @ zsoln
qoi_total_error_estimate= r.T@ phi

#### Calculate the effectivity ratio

In [298]:
effectivity_ratio_total= qoi_total_error_estimate/qoi_total_error
print(f'QoI_total_error                   = {qoi_total_error[0,0]:13.6e}')
print(f'QoI_total_error_estimate          = {qoi_total_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_total[0,0]:13.3f}')

QoI_total_error                   = -1.083184e-02
QoI_total_error_estimate          = -1.083184e-02
Effectivity ratio                 =         1.000


#### Calculate iteration error

In [299]:
qoi_iteration_error= qoi_total_error-qoi_discretization_error
print(f'QoI_iteration_error               = {qoi_iteration_error[0,0]:13.6e}')

QoI_iteration_error               =  5.171947e-05
